In [ ]:
!git clone https://bitbucket.org/ctank/swsbackend.git
!cp swsbackend/uri.py ./
!cp swsbackend/codes.py ./
!cp swsbackend/client.py ./
!ls

Cloning into 'swsbackend'...
Receiving objects: 100% (265/265), 875.59 KiB | 8.26 MiB/s, done.
Resolving deltas: 100% (145/145), done.
client.py  codes.py  sample_data  swsbackend  uri.py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1 Data Collection

In [ ]:
import client
import csv
import datetime
import pandas as pd
import numpy as np

# Log in to the client

client.login('sws3009', 'mongobongo')

# Download the data
resp = client.find_data(username='sws30095')

# Write the data as a CSV file to weather.csv
csv_header = ["timetstamp", "user", "lon", "lat", "humid", "temp", "soil_humid", "img_raw", "weather"]

print("\nWriting to weather_final.csv\n")
with open("/content/drive/MyDrive/SWS/final/weather_final_5.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(csv_header)
    
    for data in resp['data']:

        # Convert timestamp back to string
        timestamp = datetime.datetime.strftime(data['timestamp'], '%d:%m:%Y:%H:%M:%S')
        # img_raw_bytes = bytes(data['raw'], encoding='utf8')
        if data['raw']!='':
          writer.writerow([timestamp, data['username'], data['loc'][0],  data['loc'][1],  data['humid'], data['temp'], data['light'], [data['raw']],
              data['rain']])
        
print("Done!\n")
        


Writing to weather_final.csv

Done!



# 2 Data Preparation

In [ ]:
import numpy as np
import pandas as pd
import os
from PIL import Image
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


TRAIN_PERCENT = 0.8
LOOK_BACK = 10
num_epochs = 5
batch_size = 1
hidden_size = 128
skip = 1
MODEL_NAME = 'weather_pred_cpu.hd5'
# PATH = Path('/content/drive/MyDrive/SWS/final')


def imgFromBytes2Array(img_raw):
  img_raw_data = eval(img_raw)[0]
  img = Image.frombytes("RGB", (400, 225), img_raw_data)
  img = img.resize((150, 150))
  img_array = np.array(img)
  return img_array

# Generate the datasets with the given look_back.
def create_dataset(dataset_X_num, dataset_X_img, dataset_Y, look_back=10):
    dataX_num, dataX_img, dataY = [], [], []

    for i in range(len(dataset_X_num) - look_back - 1):
        a = dataset_X_num[i:(i+look_back), :]
        b = dataset_X_img[i:(i+look_back), :]
        dataX_num.append(a)
        dataX_img.append(b)
        dataY.append(dataset_Y[i + look_back, :])
    return np.array(dataX_num), np.array(dataX_img), np.array(dataY)


def load_data():
  df_1 = pd.read_csv('/content/drive/MyDrive/SWS/final/weather_final_1.csv')
  df_2 = pd.read_csv('/content/drive/MyDrive/SWS/final/weather_final_2.csv')
  # df_3 = pd.read_csv('/content/drive/MyDrive/SWS/final/weather_final_3.csv')
  # df_4 = pd.read_csv('/content/drive/MyDrive/SWS/final/weather_final_4.csv')
  df_5 = pd.read_csv('/content/drive/MyDrive/SWS/final/weather_final_5.csv')
  df = pd.concat([df_1, df_2, df_5], axis = 0)
  df['img'] = df['img_raw'].apply(imgFromBytes2Array)

  num_X = np.array(df[['humid', 'temp', 'soil_humid']]).astype('float32')
  scaler = MinMaxScaler(feature_range = (0, 1))
  num_X = scaler.fit_transform(num_X)

  img_X = np.array(df['img'].tolist()) / 255.0
  y = to_categorical(df['weather'], 3)

  # Call create_dataset to make the training and testing sets.
  num_X, img_X, y = create_dataset(num_X, img_X, y, LOOK_BACK)

  # Figure out how many vectors for training and how many for
  # testing.
  train_size = int(len(num_X) * TRAIN_PERCENT)
  val_size = len(num_X) - train_size
  # Slice the dataset accordingly

  # train_X_img, val_X_img = img_X, img_X[train_size: ]
  # train_X_num, val_X_num = num_X, num_X[train_size: ]
  # train_X_img, val_X_img = img_X[: train_size], img_X[train_size: ]
  # train_X_num, val_X_num = num_X[: train_size], num_X[train_size: ]
  # train_y, val_y = y, y[train_size: ]
  # train_y, val_y = y[: train_size], y[train_size: ]
  train_X_img, val_X_img, train_y, val_y = train_test_split(img_X, y, test_size=0.2, random_state=1)
  train_X_num, val_X_num, train_y, val_y = train_test_split(num_X, y, test_size=0.2, random_state=1)

  return train_X_num, train_X_img, train_y, val_X_num, val_X_img, val_y

In [ ]:
print(train_X_num.shape)
print(train_X_img.shape)
print(train_y.shape)
print(val_X_num.shape)
print(val_X_img.shape)
print(val_y.shape)

In [ ]:

# shape[0] gives # of rows, shape[1] gives number of columns.
# So here we have shape[0] rows each of 1 input of shape[1] columns.
# Number of columns is our lookback.

# train_X_num = np.reshape(train_X_num, (train_X_num.shape[0], 3, train_X_num.shape[1]))
# val_X_num = np.reshape(val_X_num, (val_X_num.shape[0], 3, val_X_num.shape[1]))

In [ ]:
from keras.layers import LSTM, Dense, Conv2D, Input, Flatten, TimeDistributed, MaxPooling2D, concatenate
from keras.models import Sequential, Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

def weatherPredModel(model_name):
  if os.path.exists(model_name):
    model = load_model(model_name)  

  else:  

    # define two sets of inputs
    input_num = Input(shape=(None, 3))
    input_img = Input(shape=(None, 150, 150, 3))
    
    Conv_1 = TimeDistributed(Conv2D(25, 12, strides=(2, 2), padding='valid', activation='relu'))(input_img)
    Pool_1 = TimeDistributed(MaxPooling2D(pool_size=(2,2)))(Conv_1)
    Flatten_1 = TimeDistributed(Flatten())(Pool_1)
    d = Dense(3)(Flatten_1)
    cnn_model = Model(inputs=input_img, outputs=d)

    Dense_2 = TimeDistributed(Dense(1024, activation='relu'))(input_num)
    mlp_model = Model(inputs=input_num, outputs=Dense_2)

    input_combined = concatenate([mlp_model.output, cnn_model.output])
    LSTM_1 = LSTM(hidden_size)(input_combined)
    y = Dense(3, activation='softmax')(LSTM_1)

    model = Model(inputs=[mlp_model.input, cnn_model.input], outputs=y)

  return model

def train(model, train_X_num, train_X_img, train_y,  val_X_num, val_X_img, val_y, epoch, model_name):

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # Print a summary of our network
  savemodel = ModelCheckpoint(model_name)
  stopmodel = EarlyStopping(min_delta=0.001, patience=10) 

  #Start training
  model.fit([train_X_num, train_X_img], train_y, epochs=num_epochs, validation_data = ([val_X_num, val_X_img], val_y), callbacks=[savemodel, stopmodel])


In [ ]:
train_X_num, train_X_img, train_y, val_X_num, val_X_img, val_y = load_data()
model = weatherPredModel(MODEL_NAME)
train(model, train_X_num, train_X_img, train_y, val_X_num, val_X_img, val_y, num_epochs, MODEL_NAME)

Epoch 1/5
20/20 [==============================] - 107s 4s/step - loss: 0.8688 - accuracy: 0.5632 - val_loss: 0.3101 - val_accuracy: 0.8039


INFO:tensorflow:Assets written to: weather_pred_cpu.hd5/assets


INFO:tensorflow:Assets written to: weather_pred_cpu.hd5/assets


Epoch 2/5
20/20 [==============================] - 81s 4s/step - loss: 0.3388 - accuracy: 0.8548 - val_loss: 0.0495 - val_accuracy: 0.9935


INFO:tensorflow:Assets written to: weather_pred_cpu.hd5/assets


...

INFO:tensorflow:Assets written to: weather_pred_cpu.hd5/assets


INFO:tensorflow:Assets written to: weather_pred_cpu.hd5/assets


Epoch 5/5
20/20 [==============================] - 82s 4s/step - loss: 0.0712 - accuracy: 0.9826 - val_loss: 0.0356 - val_accuracy: 1.0000


INFO:tensorflow:Assets written to: weather_pred_cpu.hd5/assets


INFO:tensorflow:Assets written to: weather_pred_cpu.hd5/assets


In [ ]:
cp -r weather_pred_cpu.hd5 /content/drive/MyDrive/SWS/final

In [ ]:
def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
	# initialize the input shape and channel dimension, assuming
	# TensorFlow/channels-last ordering
	inputShape = (height, width, depth)
	chanDim = -1
	
	# define the model input
	inputs = Input(shape=inputShape)
 
	# loop over the number of filters
	for (i, f) in enumerate(filters):
		# if this is the first CONV layer then set the input
		# appropriately
		if i == 0:
			x = inputs
 
		# CONV => RELU => BN => POOL
		x = Conv2D(f, (3, 3), padding="same")(x)
		x = Activation("relu")(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = MaxPooling2D(pool_size=(2, 2))(x)
	# flatten the volume, then FC => RELU => BN => DROPOUT
	x = Flatten()(x)
	x = Dense(16)(x)
	x = Activation("relu")(x)
	x = BatchNormalization(axis=chanDim)(x)
	x = Dropout(0.5)(x)
 
	# apply another FC layer, this one to match the number of nodes
	# coming out of the MLP
	x = Dense(4)(x)
	x = Activation("relu")(x)
 
	# check to see if the regression node should be added
	if regress:
		x = Dense(1, activation="linear")(x)
 
	# construct the CNN
	model = Model(inputs, x)
 
	# return the CNN
	return model
